# Azure AI 추론과 Phi-4 모델

> **Note:** 이 노트북은 최신 Azure AI 추론 SDK를 사용합니다. 2025년 5월 기준으로, 올바른 완료 호출 방법은 이 노트북에 표시된 대로 적절한 메시지 객체를 사용하여 `ChatCompletionsClient`를 통해 이루어집니다.

이 노트북은 Azure AI 추론 SDK를 사용하여 Azure에 호스팅된 모델, 특히 Microsoft의 Phi-4 모델을 액세스하는 방법을 보여줍니다.

## 설정 및 구성

먼저 환경을 구성하고 필요한 종속성을 설치합니다.


# Azure AI 추론과 Phi-4 모델

이 노트북은 Azure AI 추론 SDK를 사용하여 Azure에 호스팅된 모델, 특히 Microsoft의 Phi-4 모델을 사용하는 방법을 보여줍니다.

> **중요 참고:** 이 노트북은 최신 Azure AI 추론 SDK 메서드를 사용하여 완료 호출을 수행합니다. API는 다음 패턴을 따릅니다:
>
> ```python
> from azure.ai.inference import ChatCompletionsClient
> from azure.ai.inference.models import SystemMessage, UserMessage, AssistantMessage
> from azure.core.credentials import AzureKeyCredential
>
> client = ChatCompletionsClient(
>     endpoint="your_endpoint",
>     credential=AzureKeyCredential("your_api_key")
> )
>
> response = client.complete(
>     model="your_model_name",
>     messages=[SystemMessage(content="system message"), UserMessage(content="user message")],
>     temperature=0.7,
>     max_tokens=800
> )
> ```

## 설정 및 구성

먼저 환경을 구성하고 필요한 종속성을 설치합니다.


In [ ]:
# Install required packages
!pip install azure-ai-inference python-dotenv matplotlib pillow

### local.env 파일 설정하기

이 노트북을 실행하기 전에, 이 노트북과 동일한 디렉토리에 `local.env` 파일을 생성하고 아래 변수들을 추가해야 합니다:

```
# GitHub Configuration
GITHUB_TOKEN=your_personal_access_token_here
GITHUB_INFERENCE_ENDPOINT=https://models.github.ai/inference
GITHUB_MODEL=microsoft/Phi-4-reasoning

# Azure OpenAI Configuration
AZURE_API_KEY=your_azure_api_key_here
AZURE_OPENAI_ENDPOINT=your_azure_endpoint_here
AZURE_OPENAI_MODEL=Phi-4-reasoning
```

**지침:**

1. 이 노트북과 동일한 폴더에 `local.env`라는 새 파일을 만듭니다.
2. 위에 표시된 세 가지 환경 변수를 추가합니다.
3. `your_personal_access_token_here`를 GitHub 개인 액세스 토큰으로 교체합니다.
4. 더 작은 모델을 사용하려면 모델을 `microsoft/Phi-4-mini-reasoning`으로 변경할 수 있습니다.

**참고:** GitHub 토큰은 AI 모델 서비스에 접근하기 위한 적절한 권한이 필요합니다.  
**참고:** Azure API 키를 안전하게 보관하고, 공개 저장소나 노트북에 공유하지 마십시오.


## 환경 변수 로드하기

Azure AI 자격 증명이 포함된 `local.env` 파일에서 환경 변수를 로드해봅시다.


In [ ]:
import os
import json
import requests
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

# Load variables from local.env file
load_dotenv('local.env')

# Access the environment variables - using values from local.env file
api_key = os.getenv("AZURE_API")
endpoint = os.getenv("AZURE_ENDPOINT")
model_name = os.getenv("AZURE_MODEL", "Phi-4")

# Check if required environment variables are available
if not api_key or not endpoint:
    raise ValueError("AZURE_API and AZURE_ENDPOINT must be set in the local.env file.")

print(f"Azure AI Endpoint: {endpoint}")
print(f"Azure AI Model: {model_name}")
print(f"API Key available: {'Yes' if api_key else 'No'}")

# Initialize the Azure AI Inference client
try:
    # Print the endpoint for debugging
    print(f"Debug - Full endpoint URL: {endpoint}")
    
    # Create the chat completions client
    client = ChatCompletionsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(api_key)
    )
    print("Azure AI Inference client initialized successfully")
except Exception as e:
    print(f"Error initializing client: {str(e)}")
    print("If you're seeing initialization errors, please check your endpoint URL and API key.")

## 모델 추론을 위한 헬퍼 함수

Inference SDK를 사용하여 Azure AI 모델과 상호작용할 수 있는 헬퍼 함수를 만들어봅시다.


In [37]:
def generate_chat_completion(messages, temperature=0.7, max_tokens=20000, model=model_name):
    """Generate a completion using Azure AI Inference SDK's chat completions API"""
    # List of model names to try in order if the primary model fails
    model_alternatives = [model, "Phi-4", "phi-4", "gpt-4"]
    
    for attempt_model in model_alternatives:
        try:
            # Debug information
            print(f"Making API call with model: {attempt_model}")
            
            # Convert message dictionaries to proper message objects
            formatted_messages = []
            for msg in messages:
                if msg["role"] == "system":
                    formatted_messages.append(SystemMessage(content=msg["content"]))
                elif msg["role"] == "user":
                    formatted_messages.append(UserMessage(content=msg["content"]))
                elif msg["role"] == "assistant":
                    formatted_messages.append(AssistantMessage(content=msg["content"]))
            
            # Call the Azure AI Inference API
            response = client.complete(
                model=attempt_model,
                messages=formatted_messages,
                temperature=temperature,
                max_tokens=max_tokens
            )
            print(f"Success with model: {attempt_model}")
            return response.choices[0].message.content
        except Exception as e:
            error_message = str(e)
            print(f"Error during API call with {attempt_model}: {error_message}")
            
            # If this is the last model in our list, provide more detailed error information
            if attempt_model == model_alternatives[-1]:
                print("\nAll model attempts failed. Please check:")
                print("1. Your model deployment names in Azure AI Studio")
                print("2. Your API key and endpoint URL")
                print("3. That the models are actually deployed to your Azure AI resource")
                print("\nYou can modify your local.env file to use the correct model name.")
                return None
            
            # If NOT_FOUND error or other specific errors, try the next model
            if "NOT FOUND" in error_message or "model not found" in error_message.lower():
                print(f"Model {attempt_model} not found, trying alternative...")
                continue
            else:
                # For other errors, return the error
                return f"Error: {error_message}"
    
    # This line should not be reached due to the return in the final iteration above
    return None

def generate_completion(prompt, temperature=0.7, max_tokens=10000, model=model_name):
    """Generate a text completion using a simple user prompt"""
    # Simply use the chat completion function with a single user message
    return generate_chat_completion([{"role": "user", "content": prompt}], temperature, max_tokens, model)

## 예제 1: 9개의 r에 딸기가 몇 개 필요할까요?

딸기와 r에 대해 질문하며 첫 번째 추론 예제를 실행해 봅시다.


In [38]:
example1_messages = [
    {"role": "system", "content": "You are a helpful AI assistant that answers questions accurately and concisely."},
    {"role": "user", "content": "How many strawberries do I need to collect 9 r's?"}
]

print("Messages:")
for msg in example1_messages:
    print(f"{msg['role']}: {msg['content']}")
print("\nGenerating response...\n")

# Use the chat completion function with the messages
response1 = generate_chat_completion(example1_messages)
print("Response:")
print(response1)

Messages:
system: You are a helpful AI assistant that answers questions accurately and concisely.
user: How many strawberries do I need to collect 9 r's?

Generating response...

Making API call with model: Phi-4-reasoning-mbodz
Success with model: Phi-4-reasoning-mbodz
Response:
<think>User's question: "How many strawberries do I need to collect 9 r's?" It's ambiguous. Let's parse question: "How many strawberries do I need to collect 9 r's?" Possibly referring to a game or puzzle. Possibly "r's" means "red letters" or "r's" might be "r's" as in the letter "r". Possibly meaning "strawberries" might be something else. Alternatively, maybe "r's" means "r" required objects. The phrasing "collect 9 r's" might be a puzzle. Let me check: "strawberries" might be fruit in a game like "Strawberries" game? Possibly it's a riddle: "How many strawberries do I need to collect 9 r's?" Possibly it's a puzzle: "9 r's" might be in the word "strawberries" or in something else.

I need to check if there 

### 예제 1 분석

이 예제에서 모델은 "strawberry"라는 단어에 'r'이 세 개 포함되어 있다는 것을 이해해야 합니다. 다른 LLM에서 실행하면 일반적으로 이 점을 잘못 이해하는 경우가 많습니다.


## 예제 2: 수수께끼 풀기

이제 더 복잡한 예제를 시도해 봅시다 - 여러 가지 예시가 포함된 패턴 인식 수수께끼.


In [45]:
example2_messages = [
    {"role": "system", "content": "You are a helpful AI assistant that solves riddles and finds patterns in sequences."},
    {"role": "user", "content": "I will give you a riddle to solve with a few examples, and something to complete at the end"},
    {"role": "user", "content": "nuno Δημήτρης evif Issis 4"},
    {"role": "user", "content": "ntres Inez neves Margot 4"},
    {"role": "user", "content": "ndrei Jordan evlewt Μαρία 9"},
    {"role": "user", "content": "nπέντε Kang-Yuk xis-ytnewt Nubia 21"},
    {"role": "user", "content": "nπέντε Κώστας eerht-ytnewt Μανώλης 18"}, 
    {"role": "user", "content": "nminus one-point-two Satya eno Bill X."},
    {"role": "user", "content": "What is a likely completion for X that is consistent with examples above?"}
]

print("Messages:")
for msg in example2_messages:
    print(f"{msg['role']}: {msg['content'][:50]}...")
print("\nGenerating response...\n")

response2 = generate_chat_completion(example2_messages, temperature=0.2, max_tokens=30000)
print("Response:")
print(response2)

Messages:
system: You are a helpful AI assistant that solves riddles...
user: I will give you a riddle to solve with a few examp...
user: nuno Δημήτρης evif Issis 4...
user: ntres Inez neves Margot 4...
user: ndrei Jordan evlewt Μαρία 9...
user: nπέντε Kang-Yuk xis-ytnewt Nubia 21...
user: nπέντε Κώστας eerht-ytnewt Μανώλης 18...
user: nminus one-point-two Satya eno Bill X....
user: What is a likely completion for X that is consiste...

Generating response...

Making API call with model: Phi-4-reasoning-mbodz
Success with model: Phi-4-reasoning-mbodz
Response:
<think>We are given a riddle with examples. The conversation: "I will give you a riddle to solve with a few examples, and something to complete at the end". Then the examples are provided:

1. "nuno Δημήτρης evif Issis 4"
2. "ntres Inez neves Margot 4"
3. "ndrei Jordan evlewt Μαρία 9"
4. "nπέντε Kang-Yuk xis-ytnewt Nubia 21"
5. "nπέντε Κώστας eerht-ytnewt Μανώλης 18"
6. "nminus one-point-two Satya eno Bill X."

Then question: "Wh

### 예제 2 분석

이 수수께끼는 여러 언어와 숫자 표현 간의 복잡한 패턴을 인식하는 능력을 요구합니다. 모델은 다양한 언어에서의 숫자 표현, 뒤집힌 철자, 그리고 수학적 관계를 이해해야 합니다.


## 예제 만들기

Azure에 호스팅된 모델의 기능을 테스트하기 위해 직접 예제를 만들어 보세요. 프롬프트를 수정하거나 아래에 완전히 새로운 시나리오를 만들어 보세요.


In [ ]:
# Define your custom prompt here
custom_messages = [
    {"role": "system", "content": "You are a helpful AI assistant that can solve complex problems."},
    {"role": "user", "content": "Your custom prompt here"}
]

# Uncomment the lines below to run your custom prompt
# custom_response = generate_chat_completion(custom_messages)
# print("Response to custom prompt:")
# print(custom_response)

## Azure AI 추론 SDK 문제 해결

Azure AI 추론 SDK를 사용하는 데 문제가 있다면, 아래의 일반적인 문제 해결 단계를 참고하세요:

1. **엔드포인트 형식 확인**: Azure AI 추론 SDK는 `https://your-resource-name.models.ai.azure.com` 형식의 엔드포인트를 요구합니다.

2. **모델 배포 이름 확인**: 사용 중인 모델 이름이 Azure AI 리소스에 배포된 이름과 정확히 일치하는지 확인하세요. 모델 이름은 대소문자를 구분합니다.

3. **API 키 확인**: API 키가 적절한 권한을 가지고 있으며 여전히 유효한지 확인하세요.

4. **일반적인 오류**:
   - "NOT FOUND" - 모델 이름이 존재하지 않거나 잘못 입력됨
   - "Authentication failed" - API 키가 유효하지 않음
   - "Missing required arguments" - API 호출에서 매개변수 형식이 잘못됨

5. **local.env 파일 업데이트**: Azure AI 배포에 따라 올바른 값을 사용하여 local.env 파일을 업데이트하세요.

자세한 내용은 [Azure AI 추론 SDK 문서](https://learn.microsoft.com/en-gb/python/api/overview/azure/ai-inference-readme?view=azure-python-preview)를 참조하세요.


In [ ]:
# This cell performs diagnostic tests specific to Azure AI Inference SDK

print("=== Azure AI Inference SDK Diagnostics ===")

# Display the current configuration
print(f"Endpoint: {endpoint}")
print(f"Model name: {model_name}")
print(f"API key available: {'Yes' if api_key else 'No'}")

# Try a minimal test request with the AI Inference SDK
try:
    # Create minimal chat messages for testing
    test_messages = [UserMessage(content="Hello, please respond with one word: 'Working'")]
    
    print("\nAttempting test request with Azure AI Inference SDK...")
    test_response = client.complete(
        model=model_name,
        messages=test_messages,
        temperature=0.7,
        max_tokens=10
    )
    print(f"Status: Success!")
    print(f"Response: {test_response.choices[0].message.content}")
    print("\nNote: For information about available models, please check the Azure AI Studio portal where you can see all deployed models for your endpoint.")
except Exception as e:
    print(f"\nTest request failed: {str(e)}")
    print("\nSuggestions:")
    print("1. Check that your Azure AI endpoint is correctly formatted")
    print("2. Verify that the model is deployed to your Azure AI resource")
    print("3. Ensure your API key is valid")
    print("4. Try a different model name if available")

print("\n=== End of Diagnostics ===\n")

## 결론

이 노트북에서는 Azure AI Inference SDK를 사용하여 Azure에 호스팅된 AI 모델과 상호작용하는 방법을 보여주었으며, 특히 Microsoft의 Phi-4 기능을 다양한 추론 작업에서 테스트했습니다. 주요 개념은 다음과 같습니다:

1. Azure AI 엔드포인트에 대한 인증 설정
2. 채팅 형식으로 프롬프트 구성
3. 다양한 예제를 통해 모델의 추론 능력 테스트
4. 미로 탐색과 같은 복잡한 문제의 해결 방법 시각화

Azure AI Inference SDK는 다양한 Azure AI 모델에 접근할 수 있는 일관된 인터페이스를 제공하며, Microsoft로부터 지속적인 업데이트와 지원을 받을 예정입니다.



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있지만, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서를 해당 언어로 작성된 상태에서 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역을 사용함으로써 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
